In [15]:
# 📥 Bereinigten Datensatz laden & vorbereiten für Feature Engineering

import os
import pandas as pd

# Dateipfad definieren
projekt_root = os.path.abspath("../")
data_dir = os.path.join(projekt_root, "data", "raw")
filename = "sickness_table.csv"
file_path = os.path.join(data_dir, filename)

# CSV laden
df = pd.read_csv(file_path, index_col=0)

# Tippfehler korrigieren
if "dafted" in df.columns:
    df.rename(columns={"dafted": "drafted"}, inplace=True)

# Datum formatieren & setzen
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df.set_index("date", inplace=True)
df = df.asfreq("D")

# Zielvariable erzeugen
df["target_bereitschaft"] = df["sby_need"] + df["drafted"]

# Irrelevante Spalten entfernen (falls vorhanden)
drop_cols = ["sby_need", "drafted", "n_sby"]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

# Vorschau
display(df.head())

,n_sick,calls,n_duty,target_bereitschaft
date,,,,
2016-04-01,73,8154.0,1700,4.0
2016-04-02,64,8526.0,1700,70.0
2016-04-03,68,8088.0,1700,0.0
2016-04-04,71,7044.0,1700,0.0
2016-04-05,63,7236.0,1700,0.0


In [16]:
# 🕒 Zeitbasierte Merkmale generieren

import numpy as np

# Kopie zum Schutz des Originals
df_feat = df.copy()

# Grundlegende Zeitmerkmale
df_feat["weekday"] = df_feat.index.weekday
df_feat["month"] = df_feat.index.month
df_feat["year"] = df_feat.index.year
df_feat["day"] = df_feat.index.day
df_feat["quarter"] = df_feat.index.quarter
df_feat["day_of_year"] = df_feat.index.dayofyear
df_feat["weekofyear"] = df_feat.index.isocalendar().week.astype(int)

# Strukturmerkmale
df_feat["is_weekend"] = df_feat["weekday"].isin([5, 6]).astype(int)
df_feat["is_month_start"] = df_feat.index.is_month_start.astype(int)
df_feat["is_month_end"] = df_feat.index.is_month_end.astype(int)
df_feat["is_quarter_start"] = df_feat.index.is_quarter_start.astype(int)
df_feat["is_quarter_end"] = df_feat.index.is_quarter_end.astype(int)

# Saison (Winter=0, Frühling=1, Sommer=2, Herbst=3)
df_feat["season"] = (df_feat.index.month % 12) // 3

# Zyklische Kodierungen
df_feat["weekday_sin"] = np.sin(2 * np.pi * df_feat["weekday"] / 7)
df_feat["weekday_cos"] = np.cos(2 * np.pi * df_feat["weekday"] / 7)
df_feat["month_sin"] = np.sin(2 * np.pi * df_feat["month"] / 12)
df_feat["month_cos"] = np.cos(2 * np.pi * df_feat["month"] / 12)

# One-Hot-Encoding: Quartal & Saison
dummies = pd.get_dummies(df_feat[["quarter", "season"]], columns=["quarter", "season"], prefix=["quarter", "season"])
df_feat = pd.concat([df_feat.drop(columns=["quarter", "season"]), dummies], axis=1)

# Vorschau
print("✅ Zeitbasierte Features hinzugefügt.")
display(df_feat.head())

✅ Zeitbasierte Features hinzugefügt.


,n_sick,calls,n_duty,target_bereitschaft,weekday,month,year,day,day_of_year,weekofyear,...,month_sin,month_cos,quarter_1,quarter_2,quarter_3,quarter_4,season_0,season_1,season_2,season_3
date,,,,,,,,,,,,,,,,,,,,,
2016-04-01,73,8154.0,1700,4.0,4,4,2016,1,92,13,...,0.866025,-0.5,False,True,False,False,False,True,False,False
2016-04-02,64,8526.0,1700,70.0,5,4,2016,2,93,13,...,0.866025,-0.5,False,True,False,False,False,True,False,False
2016-04-03,68,8088.0,1700,0.0,6,4,2016,3,94,13,...,0.866025,-0.5,False,True,False,False,False,True,False,False
2016-04-04,71,7044.0,1700,0.0,0,4,2016,4,95,14,...,0.866025,-0.5,False,True,False,False,False,True,False,False
2016-04-05,63,7236.0,1700,0.0,1,4,2016,5,96,14,...,0.866025,-0.5,False,True,False,False,False,True,False,False


In [17]:
# 🔁 Lag-Features erzeugen für ausgewählte Spalten

# Konfiguration der Lag-Tage und Spalten
lag_days = [1, 2, 3, 31]
columns_to_lag = ["target_bereitschaft", "calls"]

# Kopie des DataFrames
df_lag = df_feat.copy()

# Lag-Features erzeugen
for col in columns_to_lag:
    for lag in lag_days:
        lagged_col = f"lag_{col}_{lag}"
        df_lag[lagged_col] = df_lag[col].shift(lag)

# Vorschau
print("✅ Lag-Features hinzugefügt:")
display(df_lag[[col for col in df_lag.columns if col.startswith("lag_")]].head())

✅ Lag-Features hinzugefügt:


,lag_target_bereitschaft_1,lag_target_bereitschaft_2,lag_target_bereitschaft_3,lag_target_bereitschaft_31,lag_calls_1,lag_calls_2,lag_calls_3,lag_calls_31
date,,,,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-02,4.0,NaN,NaN,NaN,8154.0,NaN,NaN,NaN
2016-04-03,70.0,4.0,NaN,NaN,8526.0,8154.0,NaN,NaN
2016-04-04,0.0,70.0,4.0,NaN,8088.0,8526.0,8154.0,NaN
2016-04-05,0.0,0.0,70.0,NaN,7044.0,8088.0,8526.0,NaN


In [18]:
# 📊 Rollierende Statistik-Features berechnen

# Konfiguration
windows = [7, 30]
stats = ["mean", "std", "min", "max"]
columns_to_roll = ["target_bereitschaft", "calls"]

# Kopie des vorherigen DataFrames
df_roll = df_lag.copy()

# Rollierende Features berechnen
for col in columns_to_roll:
    for window in windows:
        roll_obj = df_roll[col].rolling(window=window, min_periods=1)
        for stat in stats:
            feature_name = f"roll_{col}_{stat}_{window}"
            if stat == "mean":
                df_roll[feature_name] = roll_obj.mean()
            elif stat == "std":
                df_roll[feature_name] = roll_obj.std()
            elif stat == "min":
                df_roll[feature_name] = roll_obj.min()
            elif stat == "max":
                df_roll[feature_name] = roll_obj.max()
            else:
                raise ValueError(f"Unbekannte Statistik: {stat}")

# Vorschau der neuen Features
print("✅ Rollierende Statistik-Features hinzugefügt:")
display(df_roll[[col for col in df_roll.columns if col.startswith("roll_")]].head())

✅ Rollierende Statistik-Features hinzugefügt:


,roll_target_bereitschaft_mean_7,roll_target_bereitschaft_std_7,roll_target_bereitschaft_min_7,roll_target_bereitschaft_max_7,roll_target_bereitschaft_mean_30,roll_target_bereitschaft_std_30,roll_target_bereitschaft_min_30,roll_target_bereitschaft_max_30,roll_calls_mean_7,roll_calls_std_7,roll_calls_min_7,roll_calls_max_7,roll_calls_mean_30,roll_calls_std_30,roll_calls_min_30,roll_calls_max_30
date,,,,,,,,,,,,,,,,
2016-04-01,4.000000,NaN,4.0,4.0,4.000000,NaN,4.0,4.0,8154.0,NaN,8154.0,8154.0,8154.0,NaN,8154.0,8154.0
2016-04-02,37.000000,46.669048,4.0,70.0,37.000000,46.669048,4.0,70.0,8340.0,263.043723,8154.0,8526.0,8340.0,263.043723,8154.0,8526.0
2016-04-03,24.666667,39.310728,0.0,70.0,24.666667,39.310728,0.0,70.0,8256.0,236.144024,8088.0,8526.0,8256.0,236.144024,8088.0,8526.0
2016-04-04,18.500000,34.385074,0.0,70.0,18.500000,34.385074,0.0,70.0,7953.0,635.933959,7044.0,8526.0,7953.0,635.933959,7044.0,8526.0
2016-04-05,14.800000,30.906310,0.0,70.0,14.800000,30.906310,0.0,70.0,7809.6,637.280786,7044.0,8526.0,7809.6,637.280786,7044.0,8526.0


In [19]:
# 📅 Feiertagsliste als DataFrame erzeugen

from dateutil.easter import easter

# Zeitraum bestimmen
start_year = df_roll.index.min().year
end_year = df_roll.index.max().year

# Liste für Feiertagseinträge
holiday_rows = []

# Feiertagsdefinition
for year in range(start_year, end_year + 1):
    easter_sunday = easter(year)

    def add(name, date, typ="all"):
        holiday_rows.append({
            "name": name,
            "date": pd.to_datetime(date),
            "type": typ
        })

    # Fixe Feiertage
    add("Neujahr", f"{year}-01-01")
    add("Heilige Drei Könige", f"{year}-01-06")
    add("Valentinstag", f"{year}-02-14")
    add("Internationaler Frauentag", f"{year}-03-08")
    add("Tag der Arbeit", f"{year}-05-01", typ="signifikant")
    add("Mariä Himmelfahrt", f"{year}-08-15")
    add("Tag der Deutschen Einheit", f"{year}-10-03")
    add("Reformationstag", f"{year}-10-31")
    add("Allerheiligen", f"{year}-11-01", typ="signifikant")
    add("Heiligabend", f"{year}-12-24")
    add("Erster Weihnachtsfeiertag", f"{year}-12-25")
    add("Zweiter Weihnachtsfeiertag", f"{year}-12-26")
    add("Silvester", f"{year}-12-31")

    # Bewegliche Feiertage
    add("Ostersonntag", easter_sunday)
    add("Rosenmontag", easter_sunday - pd.Timedelta(days=48))
    add("Gründonnerstag", easter_sunday - pd.Timedelta(days=3))
    add("Karfreitag", easter_sunday - pd.Timedelta(days=2))
    add("Ostermontag", easter_sunday + pd.Timedelta(days=1), typ="signifikant")
    add("Christi Himmelfahrt", easter_sunday + pd.Timedelta(days=39))
    add("Pfingstsonntag", easter_sunday + pd.Timedelta(days=49))
    add("Pfingstmontag", easter_sunday + pd.Timedelta(days=50))
    add("Fronleichnam", easter_sunday + pd.Timedelta(days=60))
    add("Volkstrauertag", easter_sunday + pd.Timedelta(days=35), typ="signifikant")
    add("Buß- und Bettag", easter_sunday + pd.Timedelta(days=42))

# Feiertagsliste als DataFrame
holiday_df = pd.DataFrame(holiday_rows)

In [20]:
# 🎯 Feiertagsfeatures auf df anwenden

df_feat_holiday = df_roll.copy()
idx = df_feat_holiday.index

# Feiertage nach Typ
all_dates = holiday_df[holiday_df["type"] == "all"]["date"]
sig_dates = holiday_df[holiday_df["type"] == "signifikant"]["date"]

# Binäre Feiertagsindikatoren
for offset, label in [(-1, "before"), (0, ""), (1, "after")]:
    df_feat_holiday[f"holiday_{label}".strip("_")] = idx.isin(all_dates + pd.Timedelta(days=offset)).astype(int)
    df_feat_holiday[f"holiday_significant_{label}".strip("_")] = idx.isin(sig_dates + pd.Timedelta(days=offset)).astype(int)

# Feiertagsfenster (0 = kein Bezug, 1 = vorher, 2 = Feiertag, 3 = danach)
df_feat_holiday["holiday_window"] = (
    df_feat_holiday["holiday_before"] * 1 +
    df_feat_holiday["holiday"] * 2 +
    df_feat_holiday["holiday_after"] * 3
)

# Feiertagsdichte ±3 Tage (7-Tage-Rollfenster)
holiday_flags = idx.isin(all_dates).astype(int)
holiday_density = pd.Series(holiday_flags, index=idx).rolling(window=7, center=True, min_periods=1).sum()
df_feat_holiday["holiday_density_7d"] = holiday_density

# Feiertagscluster
df_feat_holiday["holiday_cluster"] = (df_feat_holiday["holiday_density_7d"] >= 2).astype(int)

# Brückentag: Feiertag an Montag oder Freitag
is_mo_fr = idx.weekday.isin([0, 4]) & idx.isin(all_dates)
df_feat_holiday["holiday_on_monday_or_friday"] = is_mo_fr.astype(int)

# Feiertag am Wochenende
is_weekend = idx.weekday.isin([5, 6]) & idx.isin(all_dates)
df_feat_holiday["is_holiday_weekend"] = is_weekend.astype(int)

# Kombination: Feiertagscluster + Brückentag
df_feat_holiday["holiday_cluster_and_bruecke"] = (
    df_feat_holiday["holiday_cluster"] & df_feat_holiday["holiday_on_monday_or_friday"]
).astype(int)

# Feiertag + Wochentag als Kategorie
df_feat_holiday["holiday_and_weekday_cat"] = (
    df_feat_holiday["holiday"] * df_feat_holiday["weekday"]
).fillna(0).astype(float)

# Vorschau
print("✅ Feiertagsfeatures hinzugefügt.")
display(df_feat_holiday.filter(like="holiday").head())

✅ Feiertagsfeatures hinzugefügt.


,holiday_before,holiday_significant_before,holiday,holiday_significant,holiday_after,holiday_significant_after,holiday_window,holiday_density_7d,holiday_cluster,holiday_on_monday_or_friday,is_holiday_weekend,holiday_cluster_and_bruecke,holiday_and_weekday_cat
date,,,,,,,,,,,,,
2016-04-01,0,0,0,0,0,0,0,0.0,0,0,0,0,0.0
2016-04-02,0,0,0,0,0,0,0,0.0,0,0,0,0,0.0
2016-04-03,0,0,0,0,0,0,0,0.0,0,0,0,0,0.0
2016-04-04,0,0,0,0,0,0,0,0.0,0,0,0,0,0.0
2016-04-05,0,0,0,0,0,0,0,0.0,0,0,0,0,0.0


In [ ]:
# 💾 Finalen Feature-Datensatz speichern
output_path_features = os.path.abspath("../data/processed/sickness_table.parquet")
df_feat_holiday.to_parquet(output_path_features, index=True)

# 💾 Feiertagsliste speichern
output_path_holidays = os.path.abspath("../data/processed/holiday.parquet")
holiday_df.to_parquet(output_path_holidays, index=True)